In [ ]:
# !pip install patool
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 33.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout, GRU, ConvLSTM2D, Conv3D, Flatten, Bidirectional
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
import mediapipe as mp
# import pickle as pk
from google.colab import drive
from google.colab import files

In [ ]:
drive.mount("/content/drive", force_remount=True)
%cd drive/MyDrive/Colab Notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
# import patoolib
# patoolib.extract_archive(path, outdir="./")

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
holistic = mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=1,
    enable_segmentation=False,
    refine_face_landmarks=False,
    min_detection_confidence=0.4)

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    return  results

In [ ]:
def normalize_norm(pose, face, lh, rh):
    
    
    norm = np.sum(pose**2, axis=-1)**(1./2) + 1e-7
    tmp = pose.T/norm
    pose = tmp.T
    
    norm = np.sum(face**2, axis=-1)**(1./2) + 1e-7
    tmp = face.T/norm
    face = tmp.T
    
    norm = np.sum(lh**2, axis=-1)**(1./2) + 1e-7
    tmp = lh.T/norm
    lh = tmp.T
    
    norm = np.sum(rh**2, axis=-1)**(1./2) + 1e-7
    tmp = rh.T/norm
    rh = tmp.T
    return pose, face, lh, rh

In [ ]:
def normalize_zscore(pose, face, lh, rh):
       
    m = pose.mean(axis=0)
    std  = pose.std(axis=0) + 1e-7
    pose = (pose - m)/std
    
    # print(pose.shape,m.shape,std.shape)
    
    m = face.mean(axis=0)
    std  = face.std(axis=0) + 1e-7
    face = (face - m)/std
    
    m = lh.mean(axis=0)
    std  = lh.std(axis=0) + 1e-7
    lh = (lh - m)/std
    
    m = rh.mean(axis=0)
    std  = rh.std(axis=0) + 1e-7
    rh = (rh - m)/std
    
    return pose, face, lh, rh

In [ ]:
def extract_keypoints(results):

    pose = np.array([[res.x, res.y,res.z] for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y,res.z] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y,res.z] for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y,res.z] for res in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros(21*3)

    pose, face, lh, rh = normalize_zscore(pose, face, lh, rh)

    return np.concatenate([pose.flatten(), face.flatten(), lh.flatten(), rh.flatten()])

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

***
***
***

In [ ]:
path ="./traindata/all"
file_list = os.listdir(path)
files = file_list
files.sort()

In [ ]:
len(files)

3200

In [ ]:
files = [path+"/"+f for f in files]# 
files[0],files[1]

('./traindata/all/001_001_001.mp4', './traindata/all/001_001_002.mp4')

In [ ]:
def videoProc(path, c=1,skip=5, o=False):
    "###"
    cap = cv2.VideoCapture(path)
    success = True
    
    framecount = 35

    fnum = 0
    v = []
    
    fpsCounter = c

    for i in range(skip):
        success, frame = cap.read()

    while success:     
        success, frame = cap.read()
        
        if fpsCounter==0:
            fpsCounter=c
        else:
            fpsCounter-=1           
            continue
        if fnum>=framecount:
            break

        
        if success:  
            # frame = cv2.resize(frame, (1920,1080), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)

            eres = extract_keypoints(results)
            tmp = np.reshape(eres, (1,-1))
            v.append(tmp)

            fnum += 1
            if o:
                draw_styled_landmarks(frame, results)
                plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                plt.show()


    
    v = np.reshape(v,(fnum,-1)) if fnum >0 else[]
    
  
    if fnum<framecount and fnum>0:
        tmp = np.zeros((framecount-fnum,1629))
        v = np.concatenate((v, tmp), axis=0) 

    return v

In [ ]:
v= videoProc(files[256],c=2,o=0)
v.shape

(35, 1629)

In [ ]:
v[34].max(),v[29].min(),v[24].std(),v[24].mean()

(3.849824438815544,
 -2.3883717377619633,
 0.9605423126557638,
 2.093680252700111e-16)

In [ ]:
l=0
vids = []
labels = []
files_ = files[2200:3200]
for f in files_:
    v = videoProc(f,c=2)
    vids.append(v)
    label = f.split('/')[-1].split('_')[0] 
    labels.append(label)
    print(l)
    l+=1

    

In [ ]:
labels = [int(l)-1 for l in labels]

In [ ]:
vids = np.array(vids)
labels = np.array(labels)

In [ ]:
labels.shape,vids.shape

((1000,), (1000, 35, 1629))

In [ ]:
np.save("./vids_3.npy",vids)
np.save("./labels_3.npy",labels)

***
***
***

In [ ]:
vids1 = np.load("./traindata/vids_1.npy")
labels1 = np.load("./traindata//labels_1.npy")

vids2 = np.load("./traindata//vids_2.npy")
labels2 = np.load("./traindata//labels_2.npy")

vids3 = np.load("./traindata//vids_3.npy")
labels3 = np.load("./traindata//labels_3.npy")

In [ ]:
vids = np.concatenate([vids1,vids2,vids3],axis=0)
labels = np.concatenate([labels1,labels2,labels3],axis=0)

In [ ]:
vids.shape,labels.shape,labels.min(),labels.max()

((3200, 35, 1629), (3200,), 0, 63)

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(vids, labels, test_size = 0.15,shuffle=True,random_state=13)
X_train.shape,y_train.shape


((2720, 35, 1629), (2720,))

In [ ]:
del vids
del labels

In [ ]:
X_train[3].min(),X_train[3].max(),X_train[3].shape

(-3.0678591483563817, 3.6039564378088675, (35, 1629))

In [ ]:
y_train[0:10],y_train.min(),y_train.max(),y_train.shape

(array([ 0, 29, 33, 37, 54, 55, 48, 15,  3,  6]), 0, 63, (2720,))

In [ ]:
model_lstm_1 = Sequential([

tf.keras.Input(shape=(35,1629),),
    
LSTM(64, return_sequences=True, activation='tanh'),
Dropout(0.2),
LSTM(128, return_sequences=True, activation='tanh'),
Dropout(0.2),
LSTM(256, return_sequences=False, activation='tanh'),

# Bidirectional(GRU(64, return_sequences=True, activation='tanh')),
# Bidirectional(GRU(256, return_sequences=True, activation='tanh')),
# Bidirectional(GRU(128, return_sequences=False, activation='tanh')),


BatchNormalization(),
Dropout(0.4),



Dense(256, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,1)),  
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 

Dense(128, activation='linear',kernel_regularizer=tf.keras.regularizers.L1L2(0.01,1)),  
BatchNormalization(),#axis=-1,center=True,scale=True,
ReLU(), 


Dense(64, activation='softmax')
])


In [ ]:
model_lstm_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 35, 64)            433664    
                                                                 
 dropout_6 (Dropout)         (None, 35, 64)            0         
                                                                 
 lstm_7 (LSTM)               (None, 35, 128)           98816     
                                                                 
 dropout_7 (Dropout)         (None, 35, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 256)               394240    
                                                                 
 batch_normalization_6 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                      

In [ ]:
model_lstm_1.compile(tf.keras.optimizers.Adam(learning_rate=1e-3,beta_1=0.9,beta_2=0.999,epsilon=1e-07,)
        ,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["accuracy"])

In [ ]:
model_lstm_1.fit(X_train, y_train, epochs=256, batch_size=128, )#batch_size=64 ,#validation_data=(X_dev, y_dev)

Epoch 1/256
22/22 [==============================] - 11s 34ms/step - loss: 380.3568 - accuracy: 0.0441
Epoch 2/256
22/22 [==============================] - 1s 30ms/step - loss: 204.9093 - accuracy: 0.1261
Epoch 3/256
22/22 [==============================] - 1s 31ms/step - loss: 104.6230 - accuracy: 0.2474
Epoch 4/256
22/22 [==============================] - 1s 30ms/step - loss: 51.1315 - accuracy: 0.4437
Epoch 5/256
22/22 [==============================] - 1s 33ms/step - loss: 24.1011 - accuracy: 0.6004
Epoch 6/256
22/22 [==============================] - 1s 36ms/step - loss: 11.1571 - accuracy: 0.7287
Epoch 7/256
22/22 [==============================] - 1s 44ms/step - loss: 5.4658 - accuracy: 0.7879
Epoch 8/256
22/22 [==============================] - 1s 52ms/step - loss: 3.2975 - accuracy: 0.8202
Epoch 9/256
22/22 [==============================] - 2s 69ms/step - loss: 2.5424 - accuracy: 0.8401
Epoch 10/256
22/22 [==============================] - 1s 61ms/step - loss: 2.1869 - accura

In [ ]:
model_lstm_1.compile(tf.keras.optimizers.Adam(learning_rate=1e-4,beta_1=0.9,beta_2=0.999,epsilon=1e-07,)
        ,loss=tf.keras.losses.SparseCategoricalCrossentropy()
            ,metrics=["accuracy"])

In [ ]:
model_lstm_1.fit(X_train, y_train, epochs=128, batch_size=128,)

Epoch 1/128
22/22 [==============================] - 10s 34ms/step - loss: 0.2304 - accuracy: 0.9982
Epoch 2/128
22/22 [==============================] - 1s 30ms/step - loss: 0.1348 - accuracy: 1.0000
Epoch 3/128
22/22 [==============================] - 1s 30ms/step - loss: 0.1026 - accuracy: 0.9996
Epoch 4/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0862 - accuracy: 1.0000
Epoch 5/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0771 - accuracy: 1.0000
Epoch 6/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0730 - accuracy: 1.0000
Epoch 7/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0754 - accuracy: 0.9993
Epoch 8/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0719 - accuracy: 1.0000
Epoch 9/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0680 - accuracy: 1.0000
Epoch 10/128
22/22 [==============================] - 1s 29ms/step - loss: 0.0679 - accuracy: 0.999

In [ ]:
model_lstm_1.evaluate(X_train, y_train)

85/85 [==============================] - 1s 10ms/step - loss: 0.0408 - accuracy: 1.0000


[0.04075738787651062, 1.0]

In [ ]:
model_lstm_1.evaluate(X_dev, y_dev)

15/15 [==============================] - 0s 9ms/step - loss: 0.0673 - accuracy: 0.9958


[0.06730152666568756, 0.9958333373069763]

In [ ]:
model_lstm_1.save("./modelArgantine/v0_1")

In [ ]:
def videoProc2(path, c=1, skip=0, o=False):
    "###"
    cap = cv2.VideoCapture(path)
    success = True
    
    framecount = 35

    fnum = 0
    v = []
    
    fpsCounter = c


    while success:     
        success, frame = cap.read()
        
        if fpsCounter==0:
            fpsCounter=c
        else:
            fpsCounter-=1           
            continue
        
        
        if success:  
            frame = cv2.resize(frame, (1920,1080), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)

            eres = extract_keypoints(results)
            tmp = np.reshape(eres, (1,-1))
            v.append(tmp)

            fnum += 1
            if o:
                draw_styled_landmarks(frame, results)
                plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                plt.show()


    
    v = np.reshape(v,(fnum,-1)) if fnum >0 else[]


    if fnum > framecount+skip:
        # f = int((fnum>framecount)/2)
        v = v[skip:framecount+skip]
    else:  v[0:framecount]
        
    if fnum<framecount and fnum>0:
        tmp = np.zeros((framecount-fnum,1629))
        v = np.concatenate((v, tmp), axis=0) 
    return v

In [ ]:
path ="./traindata/all"
file_list = os.listdir(path)
files = file_list
files.sort()
files = [path+"/"+f for f in files]# 
files[0],files[1]

('./traindata/all/001_001_001.mp4', './traindata/all/001_001_002.mp4')

In [ ]:
v = videoProc2(files[512],c=0,skip=8,o=0)#files[123]
v[34].max(),v[29].max(),v[24].max(),v[15].max(),v.shape

(3.967122850833467,
 3.967190793425681,
 3.9582499032424643,
 3.9716401921654456,
 (35, 1629))

In [ ]:
p=model_lstm_1.predict(np.reshape(v, (1,35,1629)))
np.argmax(p),p


1/1 [==============================] - 0s 24ms/step


(10, array([[6.6534440e-06, 9.2004177e-07, 7.8497196e-06, 9.9774676e-05,
         1.8294284e-06, 7.0426572e-04, 6.6068409e-05, 2.4150679e-05,
         2.7859164e-04, 6.1483202e-06, 9.9753004e-01, 8.5671045e-06,
         1.6739761e-05, 4.2467935e-05, 4.0553806e-07, 6.3226551e-05,
         1.1462183e-05, 5.3659420e-07, 4.2449232e-05, 7.7958604e-05,
         4.4829380e-07, 7.6543131e-08, 3.4410614e-04, 1.4799239e-09,
         1.3731545e-08, 1.7967602e-09, 6.9691123e-06, 2.7562690e-09,
         1.3686612e-06, 8.1729352e-09, 1.0872761e-05, 3.9033015e-08,
         1.3160633e-09, 2.8876062e-05, 4.2772126e-07, 7.2412796e-08,
         1.7051432e-06, 1.4672322e-04, 4.0374939e-09, 3.5031039e-08,
         2.8967229e-08, 1.6245968e-06, 5.0383750e-07, 7.2494743e-08,
         2.4702723e-07, 6.7493957e-08, 4.5755139e-10, 8.9678597e-06,
         4.3740888e-06, 7.4908728e-09, 1.2797406e-07, 1.4423752e-07,
         2.0365169e-05, 1.7955854e-07, 1.6089983e-07, 2.6609702e-04,
         5.5141159e-06, 1.5504